In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Pandas

## Конфигурирование

In [2]:
pd.set_option('display.max_columns', 40)
# pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
#pd.describe_option()
pd.get_option('display.max_columns')

40

## Series

### Созданиe

In [4]:
cities = ['Moscow', 'Saint-Petersburg', 'Penza']
area = [123, 100, 15]
cities_area_series = pd.Series(data=area, index=cities, name='cities_area')
cities_area_series

Moscow              123
Saint-Petersburg    100
Penza                15
Name: cities_area, dtype: int64

In [5]:
cars_price = {'geely': 2_000_000, 'haval': 2.5*1E6}
pd.Series(cars_price, name='cars_price')

geely   2000000.00
haval   2500000.00
Name: cars_price, dtype: float64

### Получение данных

In [6]:
print(cities_area_series['Moscow'], cities_area_series.iloc[0])

123 123


In [7]:
cities_area_series[cities_area_series >= 100]

Moscow              123
Saint-Petersburg    100
Name: cities_area, dtype: int64

### Изменение данных

In [8]:
cities_area_series['Moscow'] = 250
print(cities_area_series['Moscow'])

250


In [9]:
cities_area_series.index=['London', 'Paris', 'Amsterdam']
cities_area_series

London       250
Paris        100
Amsterdam     15
Name: cities_area, dtype: int64

In [10]:
def increase_by_two(x):
    return x + 2

cities_area_series = cities_area_series.apply(increase_by_two)
cities_area_series

London       252
Paris        102
Amsterdam     17
Name: cities_area, dtype: int64

## DataFrame

In [11]:
stock_price = pd.DataFrame()
for ticker in ['SPCE', 'KEX', 'AAPL', 'TSLA', 'AMZN']:
    df = pd.read_csv(f"data/{ticker}_stock.csv", parse_dates=['dt'], skiprows=0)
    df['ticker'] = ticker
    if stock_price.empty:
        stock_price = df.copy()
        continue
    stock_price = pd.concat([stock_price, df])
stock_price=stock_price.reset_index(drop=True)

In [12]:
stock_price.head(), stock_price.index

(                   dt  price  change  change_percent      volume ticker
 0 2023-01-03 16:00:00   3.49    0.00            0.00  7668027.00   SPCE
 1 2023-01-04 16:00:00   3.64    0.15            0.04  4858879.00   SPCE
 2 2023-01-05 16:00:00   3.66    0.17            0.05  5262860.00   SPCE
 3 2023-01-06 16:00:00   3.69    0.20            0.06  4090692.00   SPCE
 4 2023-01-09 16:00:00   3.91    0.42            0.12 10357731.00   SPCE,
 RangeIndex(start=0, stop=1165, step=1))

### Создание новых столбцов

In [13]:
# добавление столбца с временем года на основании даты
def season(t_val):
    if t_val.month in (12,1,2):
        return 'winter'
    elif t_val.month in (3,4,5):
        return 'spring'
    elif t_val.month in (6,7,8):
        return "summer"
    else:
        return "autumn"
    

stock_price['season'] = stock_price['dt'].apply(season)
stock_price['season'].value_counts(normalize=True, dropna=False)

season
spring   0.27
summer   0.27
autumn   0.27
winter   0.18
Name: proportion, dtype: float64

In [14]:
# np.select
conditions = [(stock_price['season'] == "spring"), (stock_price['season'] == "summer"), (stock_price['season'] == "autumn"), (stock_price['season'] == "winter")]
values = ['весна', 'лето', 'осень', 'зима']

stock_price['время года'] = np.select(conditions, values)
stock_price['время года'].value_counts(normalize=True, dropna=False)

время года
весна   0.27
лето    0.27
осень   0.27
зима    0.18
Name: proportion, dtype: float64

In [15]:
# введение столбца на основании условия к другому столбцу
median_price = stock_price.price.median()
stock_price['price_more_then_median'] = ['больше медианы' if price_value > median_price \
                                         else 'равно медиане' if price_value == median_price \
                                         else 'меньше медианы' for price_value in stock_price.price] 
stock_price.price_more_then_median.value_counts(normalize=True)

price_more_then_median
меньше медианы   0.50
больше медианы   0.50
равно медиане    0.00
Name: proportion, dtype: float64

In [16]:
# введение столбца на основании условий к значений из нескольких столбцов
stock_price['is_positive_trend'] = ['позитивный тренд' if p > median_price and cp > 0 
                                    else 'неопределенность' if p <= median_price and cp > 0 
                                    else np.nan for p, cp in zip(stock_price['price'], stock_price.change_percent)]
stock_price.is_positive_trend.value_counts(normalize=True, dropna=False)

is_positive_trend
позитивный тренд   0.50
неопределенность   0.40
NaN                0.10
Name: proportion, dtype: float64

In [17]:
# np.where
stock_price['volume_2_cat'] = np.where(stock_price.volume > stock_price.volume.quantile(.9), 'Большой объем', 'Малый объем')
stock_price['volume_2_cat'].value_counts(normalize=True)

volume_2_cat
Малый объем     0.90
Большой объем   0.10
Name: proportion, dtype: float64

In [18]:
# c использование словаря
stock_price['volume_2_cat_bool'] = stock_price['volume_2_cat'].map({'Большой объем': True, 'Малый объем': False})
stock_price['volume_2_cat_bool'].value_counts(normalize=True, dropna=False)

volume_2_cat_bool
False   0.90
True    0.10
Name: proportion, dtype: float64

### Полезные методы и атрибуты

In [19]:
stock_price.head(3)

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False
1,2023-01-04 16:00:00,3.64,0.15,0.04,4858879.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
2,2023-01-05 16:00:00,3.66,0.17,0.05,5262860.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False


In [20]:
print(f"{'*'*50}\ndtype - типы данных по столбцам\n{'*'*50}\n\
{stock_price.dtypes}\n\
{'*'*50}\nshape - форма\n{'*'*50}\n\
{stock_price.shape}\n\
{'*'*50}\nsize - количество элементов\n{'*'*50}\n\
{stock_price.size}\n\
{'*'*50}\nindex - индекс\n{'*'*50}\n\
{stock_price.index}\n\
{'*'*50}\nunique - уникальные значения\n{'*'*50}\n\
{stock_price.change.unique()}\n\
{'*'*50}\nunique - количество уникальных значений\n{'*'*50}\n\
{stock_price.change.nunique()}\n\
{'*'*50}\nvalue_counts - сколько каких значений\n{'*'*50}\n\
{stock_price.price.value_counts(normalize=True)}\n\
")

**************************************************
dtype - типы данных по столбцам
**************************************************
dt                        datetime64[ns]
price                            float64
change                           float64
change_percent                   float64
volume                           float64
ticker                            object
season                            object
время года                        object
price_more_then_median            object
is_positive_trend                 object
volume_2_cat                      object
volume_2_cat_bool                   bool
dtype: object
**************************************************
shape - форма
**************************************************
(1165, 12)
**************************************************
size - количество элементов
**************************************************
13980
**************************************************
index - индекс
*******************************

### Описательные статистики

In [21]:
stock_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1165 entries, 0 to 1164
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   dt                      1165 non-null   datetime64[ns]
 1   price                   1165 non-null   float64       
 2   change                  1165 non-null   float64       
 3   change_percent          1165 non-null   float64       
 4   volume                  1160 non-null   float64       
 5   ticker                  1165 non-null   object        
 6   season                  1165 non-null   object        
 7   время года              1165 non-null   object        
 8   price_more_then_median  1165 non-null   object        
 9   is_positive_trend       1049 non-null   object        
 10  volume_2_cat            1165 non-null   object        
 11  volume_2_cat_bool       1165 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(4), objec

In [22]:
# описательные статистики для числовых данных
stock_price.describe(exclude=['datetime'])

,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
count,1165.00,1165.00,1165.00,1160.00,1165,1165,1165,1165,1049,1165,1165
unique,NaN,NaN,NaN,NaN,5,4,4,3,2,2,2
top,NaN,NaN,NaN,NaN,SPCE,spring,весна,меньше медианы,позитивный тренд,Малый объем,False
freq,NaN,NaN,NaN,NaN,233,320,320,582,580,1049,1049
mean,116.75,39.59,0.39,54677280.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,77.02,43.42,0.43,52988840.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.38,-2.72,-0.60,68536.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,70.99,5.77,0.15,7752859.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,123.56,21.63,0.32,47989936.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,176.57,56.40,0.55,77416972.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Изменение типа данных

In [23]:
print(stock_price['change'].dtype)
stock_price.change = stock_price.change.astype('object')
print(stock_price['change'].dtype)
stock_price.change = stock_price.change.astype('str')
print(stock_price['change'].dtype)
stock_price.change = stock_price.change.astype('float')
print(stock_price['change'].dtype)

float64
object
object
float64


### Пропуски

In [24]:
stock_price.iloc[21, 1:3] = np.nan

In [25]:
stock_price.isna().any()

dt                        False
price                      True
change                     True
change_percent            False
volume                     True
ticker                    False
season                    False
время года                False
price_more_then_median    False
is_positive_trend          True
volume_2_cat              False
volume_2_cat_bool         False
dtype: bool

In [26]:
stock_price.isna().sum()

dt                          0
price                       1
change                      1
change_percent              0
volume                      5
ticker                      0
season                      0
время года                  0
price_more_then_median      0
is_positive_trend         116
volume_2_cat                0
volume_2_cat_bool           0
dtype: int64

In [27]:
stock_price[stock_price.isna().values.any(axis=1)]

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False
21,2023-02-02 16:00:00,NaN,NaN,0.82,20903003.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
63,2023-04-04 16:00:00,3.43,-0.06,-0.02,20661229.00,SPCE,spring,весна,меньше медианы,NaN,Малый объем,False
64,2023-04-05 16:00:00,3.19,-0.30,-0.09,11393519.00,SPCE,spring,весна,меньше медианы,NaN,Малый объем,False
65,2023-04-06 16:00:00,3.15,-0.34,-0.10,13672138.00,SPCE,spring,весна,меньше медианы,NaN,Малый объем,False
...,...,...,...,...,...,...,...,...,...,...,...,...
931,2023-12-05 12:10:00,242.85,134.75,1.25,NaN,TSLA,winter,зима,больше медианы,позитивный тренд,Малый объем,False
932,2023-01-03 16:00:00,85.82,0.00,0.00,76706040.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False
933,2023-01-04 16:00:00,85.14,-0.68,-0.01,68885123.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False
934,2023-01-05 16:00:00,83.12,-2.70,-0.03,67930825.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False


In [28]:
stock_price.price.fillna(0, inplace=True)

In [29]:
stock_price[stock_price.isna().values.any(axis=1)]

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False
21,2023-02-02 16:00:00,0.00,NaN,0.82,20903003.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
63,2023-04-04 16:00:00,3.43,-0.06,-0.02,20661229.00,SPCE,spring,весна,меньше медианы,NaN,Малый объем,False
64,2023-04-05 16:00:00,3.19,-0.30,-0.09,11393519.00,SPCE,spring,весна,меньше медианы,NaN,Малый объем,False
65,2023-04-06 16:00:00,3.15,-0.34,-0.10,13672138.00,SPCE,spring,весна,меньше медианы,NaN,Малый объем,False
...,...,...,...,...,...,...,...,...,...,...,...,...
931,2023-12-05 12:10:00,242.85,134.75,1.25,NaN,TSLA,winter,зима,больше медианы,позитивный тренд,Малый объем,False
932,2023-01-03 16:00:00,85.82,0.00,0.00,76706040.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False
933,2023-01-04 16:00:00,85.14,-0.68,-0.01,68885123.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False
934,2023-01-05 16:00:00,83.12,-2.70,-0.03,67930825.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False


In [30]:
stock_price.change.interpolate('linear', inplace=True)

In [31]:
stock_price.dropna(axis=0).shape

(1045, 12)

In [32]:
stock_price['is_positive_trend'].isna()

0        True
1       False
2       False
3       False
4       False
        ...  
1160    False
1161    False
1162    False
1163    False
1164    False
Name: is_positive_trend, Length: 1165, dtype: bool

In [33]:
stock_price['is_positive_trend'].isnull().any()

True

### Изменение порядка следования столбцов

In [34]:
print(stock_price.columns)
new_stock = stock_price[['dt','change_percent', 'change']].copy(deep=True)
new_stock.head()

Index(['dt', 'price', 'change', 'change_percent', 'volume', 'ticker', 'season',
       'время года', 'price_more_then_median', 'is_positive_trend',
       'volume_2_cat', 'volume_2_cat_bool'],
      dtype='object')


,dt,change_percent,change
0,2023-01-03 16:00:00,0.00,0.00
1,2023-01-04 16:00:00,0.04,0.15
2,2023-01-05 16:00:00,0.05,0.17
3,2023-01-06 16:00:00,0.06,0.20
4,2023-01-09 16:00:00,0.12,0.42


### Создание DF вручную

In [35]:
test_df = {
    'test1': [1, 2, 3],
    'test2': [4, 5, 6]
}
pd.DataFrame(test_df)

,test1,test2
0,1,4
1,2,5
2,3,6


In [36]:
people_info = [dict(age=25, height=180, weight=78), dict(age=35, height=190, weight=95)]
pd.DataFrame(people_info)

,age,height,weight
0,25,180,78
1,35,190,95


In [37]:
people_info = dict(Bob=dict(age=25, height=180, weight=78), Ann=dict(age=45, height=150, weight=99))
pd.DataFrame(people_info)

,Bob,Ann
age,25,45
height,180,150
weight,78,99


In [38]:
people_names = ['Bob', 'Ann', 'Max']
people_description = [dict(age=25, height=180, weight=78), 
                      dict(age=35, height=190, weight=95),
                      dict(age=7, height=120, weight=40)]

people = pd.DataFrame(data=people_description, index=people_names)

### Индексация

In [39]:
people

,age,height,weight
Bob,25,180,78
Ann,35,190,95
Max,7,120,40


In [40]:
people.iloc[0:2]

,age,height,weight
Bob,25,180,78
Ann,35,190,95


In [41]:
people.loc['Bob':'Max']

,age,height,weight
Bob,25,180,78
Ann,35,190,95
Max,7,120,40


In [42]:
people.loc['Bob':'Ann', 'height':'weight']

,height,weight
Bob,180,78
Ann,190,95


In [43]:
stock_price.head(1)

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False


In [44]:
stock_price.iloc[2:5, 1:3]

,price,change
2,3.66,0.17
3,3.69,0.20
4,3.91,0.42


In [45]:
stock_price.loc[[2 ,3], ['change', 'price']]

,change,price
2,0.17,3.66
3,0.20,3.69


In [46]:
stock_price.loc[2:10]

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
2,2023-01-05 16:00:00,3.66,0.17,0.05,5262860.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
3,2023-01-06 16:00:00,3.69,0.20,0.06,4090692.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
4,2023-01-09 16:00:00,3.91,0.42,0.12,10357731.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
5,2023-01-10 16:00:00,4.03,0.54,0.15,7897886.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
6,2023-01-11 16:00:00,4.29,0.80,0.23,8752365.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
7,2023-01-12 16:00:00,4.62,1.13,0.32,15111957.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
8,2023-01-13 16:00:00,5.19,1.70,0.49,46228829.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
9,2023-01-17 16:00:00,5.59,2.10,0.60,20166082.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
10,2023-01-18 16:00:00,4.91,1.42,0.41,18005284.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False


In [47]:
stock_price.at[2, 'price'] # берет только 1 значение, работает быстрее

3.66

In [48]:
stock_price.iat[2, 2]

0.17

### Итерация по DF

In [49]:
for i, row in stock_price.iterrows():
    print(f'Номер строки {i}, ticker: {row.ticker}, тип: {type(row)}')
    if i == 2:
        break

Номер строки 0, ticker: SPCE, тип: <class 'pandas.core.series.Series'>
Номер строки 1, ticker: SPCE, тип: <class 'pandas.core.series.Series'>
Номер строки 2, ticker: SPCE, тип: <class 'pandas.core.series.Series'>


### Фильтрация

In [50]:
people.filter(regex='b$', axis=0)

,age,height,weight
Bob,25,180,78


In [51]:
people.filter(like=r'ag')

,age
Bob,25
Ann,35
Max,7


In [52]:
people.loc[people['height'] > 130]

,age,height,weight
Bob,25,180,78
Ann,35,190,95


In [53]:
height_and_weight_verification = (people.height > 130) & (people.weight > 80)
people.loc[height_and_weight_verification]

,age,height,weight
Ann,35,190,95


In [54]:
stock_price[stock_price['season'].isin(['summer', 'winter'])].head()

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False
1,2023-01-04 16:00:00,3.64,0.15,0.04,4858879.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
2,2023-01-05 16:00:00,3.66,0.17,0.05,5262860.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
3,2023-01-06 16:00:00,3.69,0.20,0.06,4090692.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
4,2023-01-09 16:00:00,3.91,0.42,0.12,10357731.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False


In [55]:
stock_price[~stock_price['season'].isin(['summer', 'winter'])].head()

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
39,2023-03-01 16:00:00,4.73,1.24,0.36,18461814.00,SPCE,spring,весна,меньше медианы,неопределенность,Малый объем,False
40,2023-03-02 16:00:00,5.14,1.65,0.47,11817593.00,SPCE,spring,весна,меньше медианы,неопределенность,Малый объем,False
41,2023-03-03 16:00:00,5.50,2.01,0.58,11241951.00,SPCE,spring,весна,меньше медианы,неопределенность,Малый объем,False
42,2023-03-06 16:00:00,5.41,1.92,0.55,7699815.00,SPCE,spring,весна,меньше медианы,неопределенность,Малый объем,False
43,2023-03-07 16:00:00,5.31,1.82,0.52,6220768.00,SPCE,spring,весна,меньше медианы,неопределенность,Малый объем,False


### Изменение даннных

In [56]:
people.loc[height_and_weight_verification, 'weight'] = 150
people

,age,height,weight
Bob,25,180,78
Ann,35,190,150
Max,7,120,40


In [57]:
def add_two(x):
    return x + 2

def difference_between_avg(row):
    row.age -= avg_age
    row.height -= avg_height
    row.weight -= avg_weight
    return row

people['age'].apply(lambda x: x+2)
people['weight'].map(add_two)
avg_age = people.age.mean()
avg_height = people.height.mean()
avg_weight = people.weight.mean()

people_dif = pd.DataFrame(columns=['dif_age', 'dif_height', 'dif_weight'])

In [58]:
people_dif[['dif_age', 'dif_height', 'dif_weight']] = people.apply(difference_between_avg, axis='columns')

In [59]:
people_dif

,dif_age,dif_height,dif_weight
Bob,2.67,16.67,-11.33
Ann,12.67,26.67,60.67
Max,-15.33,-43.33,-49.33


In [60]:
people_dif.applymap(lambda x: x + 100)

,dif_age,dif_height,dif_weight
Bob,102.67,116.67,88.67
Ann,112.67,126.67,160.67
Max,84.67,56.67,50.67


In [61]:
people

,age,height,weight
Bob,25,180,78
Ann,35,190,150
Max,7,120,40


### MAP

In [62]:
d = {35: 45, 7: 8, 25: 'sdf'}
people.age.map(d)


Bob    sdf
Ann     45
Max      8
Name: age, dtype: object

### Shift

In [63]:
stock_price.head(2)

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False
1,2023-01-04 16:00:00,3.64,0.15,0.04,4858879.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False


In [64]:
aapl = stock_price.loc[stock_price['ticker'] == 'AAPL']
aapl = aapl.groupby('month')[['price']].sum().reset_index()
aapl.head()

KeyError: 'month'

In [ ]:
aapl['price_shift'] = aapl.price.shift(1)
aapl

In [ ]:
aapl['price_d1'] = (aapl.price - aapl.price_shift) / aapl.price_shift
aapl

### PCTchange

In [ ]:
aapl['price_d2'] = aapl.price.pct_change(1)
aapl

### Merge

In [ ]:
spce = stock_price.loc[stock_price['ticker'] == 'SPCE']
print(spce.shape)
amzn = stock_price.loc[stock_price['ticker'] == 'AMZN']
print(amzn.shape)

spce.merge(amzn, on='dt').shape

### Работа со временем

In [ ]:
stock_price['dt'] = pd.to_datetime(stock_price.dt)
stock_price.dtypes

In [ ]:
stock_price['dt'].dt.day_of_week
stock_price['dt'].dt.day
stock_price['dt'].dt.year
stock_price['dt'].dt.month
stock_price['dt'].dt.day_of_year
stock_price['dt'].dt.day_of_week
stock_price['dt'].dt.days_in_month
stock_price['dt'].dt.day_name()

In [ ]:
stock_price['dt'].dt.strftime("%d.%m.%Y %M:%H:%S")

In [ ]:
stock_price['date'] = stock_price['dt'].dt.date

In [ ]:
stock_price['time'] = stock_price.dt.dt.time

In [ ]:
stock_price['month'] = stock_price.dt.dt.month

In [ ]:
stock_price['week'] = stock_price.dt.dt.isocalendar().week

In [76]:
stock_price.head(10)

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool
0,2023-01-03 16:00:00,3.49,0.00,0.00,7668027.00,SPCE,winter,зима,меньше медианы,NaN,Малый объем,False
1,2023-01-04 16:00:00,3.64,0.15,0.04,4858879.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
2,2023-01-05 16:00:00,3.66,0.17,0.05,5262860.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
3,2023-01-06 16:00:00,3.69,0.20,0.06,4090692.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
4,2023-01-09 16:00:00,3.91,0.42,0.12,10357731.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
5,2023-01-10 16:00:00,4.03,0.54,0.15,7897886.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
6,2023-01-11 16:00:00,4.29,0.80,0.23,8752365.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
7,2023-01-12 16:00:00,4.62,1.13,0.32,15111957.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
8,2023-01-13 16:00:00,5.19,1.70,0.49,46228829.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False
9,2023-01-17 16:00:00,5.59,2.10,0.60,20166082.00,SPCE,winter,зима,меньше медианы,неопределенность,Малый объем,False


#### Группировка по времени

In [82]:
aapl.set_index('dt')['volume'].resample('M').sum()

dt
2023-01-31   1443652725.00
2023-02-28   1307294493.00
2023-03-31   1520461315.00
2023-04-30    969776368.00
2023-05-31   1275364523.00
2023-06-30   1297863403.00
2023-07-31    996368613.00
2023-08-31   1323817340.00
2023-09-30   1337873796.00
2023-10-31   1172845791.00
2023-11-30   1099760711.00
2023-12-31     89094342.00
Freq: M, Name: volume, dtype: float64

In [84]:
aapl.set_index('dt')['volume'].resample('Y').sum()

dt
2023-12-31   13834173420.00
Freq: A-DEC, Name: volume, dtype: float64

In [85]:
aapl.set_index('dt')['volume'].resample('D').sum()

dt
2023-01-03   112117471.00
2023-01-04    89113633.00
2023-01-05    80962708.00
2023-01-06    87754715.00
2023-01-07           0.00
                 ...     
2023-12-01    45704823.00
2023-12-02           0.00
2023-12-03           0.00
2023-12-04    43389519.00
2023-12-05           0.00
Freq: D, Name: volume, Length: 337, dtype: float64

### Группировка

In [ ]:
people.groupby('age').mean()

In [ ]:
people.groupby('age')['height'].agg(['min', 'max'])

In [ ]:
people.groupby('age').agg({'height': 'max', 'weight': 'min'})

In [ ]:
def square_calc(x):
    return x**2

people.groupby('age').agg(square_calc)

In [ ]:
stock_price.groupby('month')['price'].agg(['sum', 'count'])

In [ ]:
mult_ind_df = stock_price.groupby(['month', 'week'])[['price', 'volume_2_cat']].agg(
    {
        'price': ['sum', 'mean'],
        'volume_2_cat': ['count', lambda x: x]
    }
)
mult_ind_df.head()

In [ ]:
stock_price.groupby(['month', 'week'])[['volume', 'price']].apply(lambda row: sum(row['volume'] * row['price'])).head(10)

In [ ]:
import numpy as np
stock_price.groupby(['ticker', 'month'])[['volume', 'price']].agg(
    {
        'volume': [np.mean, np.median],
        'price': [np.max, np.min]        
    }
).head()

### Мультииндекс

In [ ]:
mult_ind_df = stock_price.groupby(['month', 'week'])[['price', 'volume_2_cat']].agg(
    {
        'price': ['sum', 'mean'],
        'volume_2_cat': ['count', lambda x: x]
    }
)
mult_ind_df.head()

In [ ]:
mult_ind_df.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in mult_ind_df.columns]

In [ ]:
mult_ind_df.head()

In [ ]:
df=stock_price.pivot_table(values='price', index=['season', 'ticker'], columns=['month'], aggfunc=sum)
df=df.reset_index()
df.head()

In [ ]:
df.index, df.columns
# df.columns = ['season', 'ticker', ('min', 'price'), ('max', 'price')]
# df

In [ ]:
df.melt(id_vars=['season', 'ticker'], value_vars=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

### Биннинг данных

In [69]:
aapl['volume'].min(), aapl.volume.max()

(24048344.0, 154357337.0)

In [73]:
aapl['volume_bins_cut'] = pd.cut(aapl.volume, bins=[20E6, 50E6, 100E6, 150E6], labels=['A', 'B', 'C'])
aapl['volume_bins_cut'].value_counts()

volume_bins_cut
B    151
A     72
C      8
Name: count, dtype: int64

In [75]:
aapl['volume_bins_quant'] = pd.qcut(aapl.volume, q=3, labels=['A', 'B', 'C'])
aapl['volume_bins_quant'].value_counts()

volume_bins_quant
A    78
B    77
C    77
Name: count, dtype: int64

### Корреляции

In [ ]:
plt.rcParams['figure.figsize'] = (16, 5)
# plt.figure(figsize=(16,5))
number_columns = stock_price.select_dtypes('number').columns
sns.heatmap(stock_price[number_columns].corr(), vmax=1, vmin=-1, annot=True)

### Сводные таблицы

In [ ]:
df = stock_price.groupby(['ticker', 'season'])[['volume', 'price']].agg(np.mean).reset_index()
df

In [ ]:
df.pivot_table(values=['price', 'volume'], index=['season'], columns=['ticker'])

In [ ]:
df = stock_price.pivot_table(values=['price', 'volume'], index='season', columns=['ticker'], aggfunc=[np.min, np.max])
# df.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in df.columns]
# df = df.reset_index()
df

### Визуализации

In [ ]:
stock_price.season.hist()

In [ ]:
month_price_sum = stock_price.groupby('month').price.agg(price_sum='sum')
sns.boxplot(month_price_sum, orient='h')

In [ ]:
df = stock_price.groupby(['ticker', 'month'])[['volume', 'price']].agg(
    {
        'volume': [np.mean, np.median],
        'price': [np.max, np.min]        
    }
)
df = df.reset_index()

df.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in df.columns]
df.head()

sns.scatterplot(x=df['month_'], y=df['volume_mean'], hue=df.ticker_)

In [ ]:
sns.jointplot(data=df, x='month_', y='volume_mean', hue=df.ticker_)

In [ ]:
sns.barplot(data=df, x='month_', y='price_max', hue=df.ticker_, fill=False)

### Оконные функции

In [ ]:
df = stock_price.groupby(['month', 'ticker']).agg({'price': sum})
df.head(7)

In [ ]:
df = stock_price.groupby(['ticker', 'month']).agg({'price': sum})
df.head(13)

In [ ]:
df.rolling(2).sum().head(7)

In [ ]:
df['change'] = df['price'].rolling(2, min_periods=1).apply(lambda x: round((x[1] - x[0]) * 100 / x[0], 2) if len(x) == 2 else 0)
df['change'] = df['change'].map(str)
df['change']

### Интересные методы

In [ ]:
stock_price.volume.astype('str').replace('\\.0', '!!', regex=True)

### Сравнение DF

In [ ]:
people.equals(people)

In [ ]:
(people == people).all()

### Выбор случайных записей

In [ ]:
stock_price.sample(100)

### Сохранение DF

In [ ]:
tickers = stock_price.ticker.unique().tolist()
tickers

In [ ]:
#!pip install openpyxl -y -q
from pathlib import Path
Path.mkdir(Path('test_save_df'), exist_ok=True)
for t in tickers:
    print(t)
    tdf = stock_price.loc[stock_price['ticker'] == ticker]
    tdf.to_excel(f'test_save_df/{t}.xlsx', sheet_name=ticker, index=False)

### Загрузка DF

In [86]:
import glob
df_list = []
for f in glob.glob('test_save_df/*.xlsx'):
    df_list.append(pd.read_excel(f))

df = pd.concat(df_list, ignore_index=True)
df

,dt,price,change,change_percent,volume,ticker,season,время года,price_more_then_median,is_positive_trend,volume_2_cat,volume_2_cat_bool,date,time,month,week
0,2023-01-03 16:00:00,85.82,0.00,0.00,76706040.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False,2023-01-03,16:00:00,1,1
1,2023-01-04 16:00:00,85.14,-0.68,-0.01,68885123.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False,2023-01-04,16:00:00,1,1
2,2023-01-05 16:00:00,83.12,-2.70,-0.03,67930825.00,AMZN,winter,зима,меньше медианы,NaN,Малый объем,False,2023-01-05,16:00:00,1,1
3,2023-01-06 16:00:00,86.08,0.26,0.00,83303361.00,AMZN,winter,зима,меньше медианы,неопределенность,Малый объем,False,2023-01-06,16:00:00,1,1
4,2023-01-09 16:00:00,87.36,1.54,0.02,65266056.00,AMZN,winter,зима,меньше медианы,неопределенность,Малый объем,False,2023-01-09,16:00:00,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,2023-11-29 16:00:00,146.32,60.50,0.70,40610907.00,AMZN,autumn,осень,больше медианы,позитивный тренд,Малый объем,False,2023-11-29,16:00:00,11,48
1161,2023-11-30 16:00:00,146.09,60.27,0.70,65814022.00,AMZN,autumn,осень,больше медианы,позитивный тренд,Малый объем,False,2023-11-30,16:00:00,11,48
1162,2023-12-01 16:00:00,147.03,61.21,0.71,39951833.00,AMZN,winter,зима,больше медианы,позитивный тренд,Малый объем,False,2023-12-01,16:00:00,12,48
1163,2023-12-04 16:00:00,144.84,59.02,0.69,48294244.00,AMZN,winter,зима,больше медианы,позитивный тренд,Малый объем,False,2023-12-04,16:00:00,12,49


### Pipes

In [99]:
def concat_str(df):
    df['ru_eng_season'] = df['season'] + df['время года']
    return df

def pct_change_price(df):
    df['volume_pct_change'] = df['price'].pct_change(2).fillna(0)
    return df

test = aapl[['price', 'season', 'время года']].copy()
test = test.pipe(concat_str).pipe(pct_change_price)
test




,price,season,время года,ru_eng_season,volume_pct_change
466,125.07,winter,зима,winterзима,0.00
467,126.36,winter,зима,winterзима,0.00
468,125.02,winter,зима,winterзима,-0.00
469,129.62,winter,зима,winterзима,0.03
470,130.15,winter,зима,winterзима,0.04
...,...,...,...,...,...
694,189.37,autumn,осень,autumnосень,-0.00
695,189.95,autumn,осень,autumnосень,-0.00
696,191.24,winter,зима,winterзима,0.01
697,189.43,winter,зима,winterзима,-0.00
